In [77]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate, FewShotPromptTemplate
chat=ChatOpenAI(temperature=0.1)


In [69]:
examples = [
    {
        "film": "Dune2",
        "answer": """
        Here is what I know:
        Director: Denis Villeneuve
        Main Cast: Timothée Chalamet (Paul Atreides), Rebecca Ferguson (Lady Jessica)
        Budget: $165 million
        Box Office: $434.8 million
        Genre: Epic science fiction
        Synopsis: The film is set in a distant future where Paul Atreides, whose family, the noble House Atreides, becomes embroiled in a war for the desert planet Arrakis. This planet is the sole source of "spice," a substance that grants vitality and psychic abilities, making it crucial for space travel. The story explores themes of power, betrayal, and destiny, as Paul confronts his role in a larger universe and a prophecy that might alter the course of history.
        """,
    },
    {
        "film": "Parasite",
        "answer": """
        I know this:
        Director: Bong Joon-ho
        Main Cast: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Choi Woo-shik,
        Budget: Approximately $15.5 million
        Box Office: Around $262.7 million
        Genre: Dark comedy thriller
        Synopsis: The story follows a poor family, the Kims, who cunningly embed themselves into the lives of the wealthy Park family, leading to unexpected consequences.
        """,
    },
    {
        "film": "The Lord of the Rings: The Fellowship of the Ring",
        "answer": """
        I know this:
        Director: Peter Jackson
        Main Cast:Elijah WoodIan McKellenOrlando Bloom
        Budget: $93 million
        Box Office:$883 million
        Genre: Action Adventure Drama Fantasy
        Synopsis: Tasked with the destruction of the One Ring, their quest begins to save Middle-earth from the dark powers of Sauron, marking the first installment in the "The Lord of the Rings" trilogy.
        """,
    },
]

In [74]:
example_prompt = PromptTemplate.from_template(
  "Human: {film} \n AI:{answer}"
) 

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples = examples, 
  input_variables=["film"],
  suffix="Human:What do you know about {film}?"
)
prompt.format(film="Die Hard")
chain = prompt | chat
chain.invoke({
  "film":"harry Potter"
})

AIMessage(content='AI:\n        I know this:\n        Director: Chris Columbus (1st and 2nd films), Alfonso Cuarón (3rd film), Mike Newell (4th film), David Yates (5th-8th films)\n        Main Cast: Daniel Radcliffe, Emma Watson, Rupert Grint\n        Budget: Varies per film, ranging from $125 million to $250 million\n        Box Office: Total franchise box office gross is over $7.7 billion\n        Genre: Fantasy\n        Synopsis: The series follows the journey of a young wizard, Harry Potter, as he attends Hogwarts School of Witchcraft and Wizardry, battles the dark wizard Lord Voldemort, and discovers his true destiny.')

In [82]:
example_prompt_new= ChatPromptTemplate.from_messages(
  [("human", "What do you know about {film}?"),
  ("ai", "{answer}")]
)

prompt = FewShotChatMessagePromptTemplate(
  example_prompt= example_prompt_new,
  examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages(
  [("system","You are a movie source"),
  prompt, 
  ("human", "What do you know about {film}?"),]
)
chain = final_prompt | chat 
chain.invoke({"film":"harry Potter"})

AIMessage(content='\n        I know this:\n        Director: Various directors across the series, including Chris Columbus, Alfonso Cuarón, Mike Newell, and David Yates\n        Main Cast: Daniel Radcliffe, Emma Watson, Rupert Grint\n        Budget: Varies per film, with the total budget for the series estimated at around $1.2 billion\n        Box Office: The Harry Potter film series grossed over $7.7 billion worldwide\n        Genre: Fantasy, Adventure\n        Synopsis: The series follows the journey of a young wizard, Harry Potter, as he attends Hogwarts School of Witchcraft and Wizardry, battles the dark wizard Lord Voldemort, and discovers his destiny in the wizarding world.\n        ')